In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from pandas import Series,DataFrame

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
%matplotlib inline

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split

In [ ]:
random_seed = 1111

In [ ]:
train_x_raw = pd.read_csv("G:/Redhat_share/ML_micro/train_x.csv")
train_y_raw = pd.read_csv("G:/Redhat_share/ML_micro/train_y.csv")
train_xy_raw = pd.merge(train_x_raw,train_y_raw,on='uid')
test_x = pd.read_csv("G:/Redhat_share/ML_micro/test_x.csv")
features_type = pd.read_csv("G:/Redhat_share/ML_micro/features_type.csv")
test_uid = test_x.uid
train_xy_raw.head()

In [ ]:
train_xy = train_xy_raw.drop(['uid'],axis=1)
train,val = train_test_split(train_xy_raw, test_size = 0.2,random_state=random_seed)
train_y = train.y
train_x = train.drop(['y'],axis=1)
val_y = val.y
val_x = val.drop(['y'],axis=1)

In [ ]:
dtrain = xgb.DMatrix(train_x, label=train_y)
dval = xgb.DMatrix(val_x,label=val_y)
dtest = xgb.DMatrix(test_x)
params={
    'booster':'gbtree',
    'objective': 'binary:logistic',
    'early_stopping_rounds':100,
    'scale_pos_weight': 5,
        'eval_metric': 'auc',
    'gamma':0.05,
#    'max_depth':8,
    'lambda':600,
        'subsample':0.6,
        'colsample_bytree':0.4,
        'min_child_weight':3,
       # 'eta': 0.02,
    'seed':random_seed,
    'nthread':8
    }

In [ ]:
watchlist  = [(dval,'val'), (dtrain,'train')]
model = xgb.train(params,dtrain,num_boost_round=2000,evals=watchlist)
model.save_model('xgb.model')
#预测
test_y = model.predict(dtest,ntree_limit=model.best_ntree_limit)
test_result = pd.DataFrame(columns=["uid","score"])
test_result.uid = test_uid
test_result.score = test_y
test_result.to_csv("result.csv",index=None,encoding='utf-8')